In [1]:
import pandas as pd
import numpy as np

In [2]:
# Clientes
clientes = pd.DataFrame({
    'cliente_id': range(1, 6),
    'nombre': ['Ana', 'Juan', 'María', 'Pedro', 'Laura'],
    'segmento': ['Premium', 'Regular', 'Premium', 'Regular', 'VIP']
})

In [3]:
# Pedidos
pedidos = pd.DataFrame({
    'pedido_id': range(1, 11),
    'cliente_id': np.random.choice(range(1, 6), 10),
    'producto': np.random.choice(['A', 'B', 'C', 'D'], 10),
    'precio': np.random.uniform(50, 500, 10).round(2),
    'fecha': pd.date_range('2024-01-01', periods=10)
})

In [4]:
print("Clientes:")
display(clientes)

print("\nPedidos:")
display(pedidos)

Clientes:


,cliente_id,nombre,segmento
0,1,Ana,Premium
1,2,Juan,Regular
2,3,María,Premium
3,4,Pedro,Regular
4,5,Laura,VIP



Pedidos:


,pedido_id,cliente_id,producto,precio,fecha
0,1,4,A,86.46,2024-01-01
1,2,1,C,465.08,2024-01-02
2,3,4,D,353.43,2024-01-03
3,4,3,A,237.52,2024-01-04
4,5,4,B,176.18,2024-01-05
5,6,4,D,423.01,2024-01-06
6,7,2,B,484.45,2024-01-07
7,8,2,B,52.63,2024-01-08
8,9,2,B,307.12,2024-01-09
9,10,4,D,64.90,2024-01-10


In [5]:
# Unir pedidos con información de clientes
pedidos_enriquecidos = pd.merge(
    pedidos,
    clientes,
    on='cliente_id',
    how='left'
)

In [6]:
print("Pedidos enriquecidos:")
display(pedidos_enriquecidos)

Pedidos enriquecidos:


,pedido_id,cliente_id,producto,precio,fecha,nombre,segmento
0,1,4,A,86.46,2024-01-01,Pedro,Regular
1,2,1,C,465.08,2024-01-02,Ana,Premium
2,3,4,D,353.43,2024-01-03,Pedro,Regular
3,4,3,A,237.52,2024-01-04,María,Premium
4,5,4,B,176.18,2024-01-05,Pedro,Regular
5,6,4,D,423.01,2024-01-06,Pedro,Regular
6,7,2,B,484.45,2024-01-07,Juan,Regular
7,8,2,B,52.63,2024-01-08,Juan,Regular
8,9,2,B,307.12,2024-01-09,Juan,Regular
9,10,4,D,64.90,2024-01-10,Pedro,Regular


In [7]:
# Calcular métricas por cliente
metricas_cliente = pedidos_enriquecidos.groupby(
    ['cliente_id', 'nombre', 'segmento']
).agg({
    'pedido_id': 'count',
    'precio': ['sum', 'mean', 'max'],
    'fecha': 'max'
}).round(2)

In [8]:
# Aplanar columnas multi-nivel
metricas_cliente.columns = [
    'num_pedidos',
    'total_gastado',
    'gasto_promedio',
    'gasto_maximo',
    'ultima_compra'
]

metricas_cliente = metricas_cliente.reset_index()

In [9]:
print("Métricas por cliente:")
display(metricas_cliente)

Métricas por cliente:


,cliente_id,nombre,segmento,num_pedidos,total_gastado,gasto_promedio,gasto_maximo,ultima_compra
0,1,Ana,Premium,1,465.08,465.08,465.08,2024-01-02
1,2,Juan,Regular,3,844.20,281.40,484.45,2024-01-09
2,3,María,Premium,1,237.52,237.52,237.52,2024-01-04
3,4,Pedro,Regular,5,1103.98,220.80,423.01,2024-01-10


In [10]:
def validar_reglas_negocio(df):
    validaciones = []
    
    # VIP deben tener al menos 2 pedidos
    vip_insuficientes = df[
        (df['segmento'] == 'VIP') & (df['num_pedidos'] < 2)
    ]
    if len(vip_insuficientes) > 0:
        validaciones.append(
            f"VIPs con pocos pedidos: {len(vip_insuficientes)}"
        )
    
    # Premium no deben exceder gasto máximo
    premium_excesivos = df[
        (df['segmento'] == 'Premium') & (df['gasto_maximo'] > 800)
    ]
    if len(premium_excesivos) > 0:
        validaciones.append(
            f"Premiums con gastos excesivos: {len(premium_excesivos)}"
        )
    
    return validaciones

In [11]:
reglas_incumplidas = validar_reglas_negocio(metricas_cliente)

print("Reglas de negocio incumplidas:")
print(reglas_incumplidas)

Reglas de negocio incumplidas:
[]


In [12]:
import pandas as pd

# Crear diccionario con validaciones
resultado_validaciones = {
    "regla": [
        "VIP con al menos 2 pedidos",
        "Premium con gasto máximo <= 800"
    ],
    "cumple": [
        not ((metricas_cliente['segmento'] == 'VIP') & 
             (metricas_cliente['num_pedidos'] < 2)).any(),
        not ((metricas_cliente['segmento'] == 'Premium') & 
             (metricas_cliente['gasto_maximo'] > 800)).any()
    ]
}

validaciones_df = pd.DataFrame(resultado_validaciones)

# Crear archivo Excel de evidencia
ruta_excel = "evidencia_transformaciones_avanzadas_dia3.xlsx"

with pd.ExcelWriter(ruta_excel, engine="openpyxl") as writer:
    clientes.to_excel(writer, sheet_name="clientes", index=False)
    pedidos.to_excel(writer, sheet_name="pedidos", index=False)
    pedidos_enriquecidos.to_excel(writer, sheet_name="pedidos_enriquecidos", index=False)
    metricas_cliente.to_excel(writer, sheet_name="metricas_cliente", index=False)
    validaciones_df.to_excel(writer, sheet_name="validaciones_reglas_negocio", index=False)

print("✅ Evidencia del Día 3 generada correctamente")
print(f"Archivo creado: {ruta_excel}")

✅ Evidencia del Día 3 generada correctamente
Archivo creado: evidencia_transformaciones_avanzadas_dia3.xlsx
